[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/21--no-sql-database-mongo/08--document-database-mongo__graphs.ipynb)

In [ ]:
!sudo apt install mongodb
!sudo service mongodb start

```json
{
     "_id" : ObjectId("5b107bec1d2952d0da904dd7"),
     "title" : "Titan A.E.",
     "year" : 2000,
     "rated" : "PG",
     "runtime" : 94,
     "countries" : [
             "USA"
     ],
     "genres" : [
             "Animation",
             "Action",
             "Adventure"
     ],
     "director" : "Don Bluth, Gary Goldman, Art Vitello",
     "writers" : [
             "Hans Bauer",
             "Randall McCormick",
             "Ben Edlund",
             "John August",
             "Joss Whedon"
     ],
     "actors" : [
             "Matt Damon",
             "Bill Pullman",
             "John Leguizamo",
             "Nathan Lane"
     ],
     "plot" : "A young man learns that he has to find a hidden Earth ship before an enemy alien species does in order to secure the survival of humanity.",
     "poster" : "http://ia.media-imdb.com/images/M/MV5BMjE0NTU0ODg4NV5BMl5BanBnXkFtZTcwNzY3MTQyMQ@@._V1_SX300.jpg",
     "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     },
     "tomato" : {
             "meter" : 52,
             "image" : "rotten",
             "rating" : 5.7,
             "reviews" : 99,
             "fresh" : 51,
             "consensus" : "Great visuals, but the story feels like a cut-and-paste job of other sci-fi movies.",
             "userMeter" : 60,
             "userRating" : 3.2,
             "userReviews" : 69055
     },
     "metacritic" : 48,
     "awards" : {
             "wins" : 1,
             "nominations" : 7,
             "text" : "1 win & 7 nominations."
     },
     "type" : "movie"
}
```

In [ ]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection movies var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection films var/video_movies.bson


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint as pp
%matplotlib inline

In [ ]:
tips = sns.load_dataset("tips")
sns.relplot(x="total_bill", y="tip", data=tips);

In [ ]:
pp(type(tips))

In [ ]:
pip install pymongo

In [ ]:
import pandas as pd
from pymongo import MongoClient

In [ ]:
client = MongoClient('mongodb://localhost:27017')

In [ ]:
db = client.datascience

In [ ]:
data = list(db.movies.aggregate([
    {"$match": {"imdb.rating": {"$exists": "1", "$gte": 0, "$lte": 10}}},
    {"$project": {'rating': '$imdb.rating'}},
    {"$group": {"_id": "$rating", "count": {"$sum": 1}}},
    {"$sort": {"_id": 1}}
]))

In [ ]:
df = pd.DataFrame(data)
df = df.rename(columns={"_id": "rating"})

In [ ]:
df

In [ ]:
sns.relplot(x="rating", y="count", data=df,  kind="line");

#avarage user rating, runtime

In [ ]:
data = list(db.movies.aggregate([
    {
        "$match": 
         {
            "imdb.rating": {"$exists": "1", "$gte": 0, "$lte": 10},
            "runtime": {"$exists": "1"}
        },
    },
    {"$project": {'rating': '$imdb.rating', "runtime": "$runtime"}},
    {"$sort": {"_id": 1}}
]))

df = pd.DataFrame(data)



In [ ]:
sns.relplot(x="rating", y="runtime", data=df);

Avg rating to votes count

In [ ]:
data = list(db.movies.aggregate([
    {"$match": {
        "imdb.rating": {"$exists": "1", "$gte": 0, "$lte": 10},
        "year": {"$gte": 2000}
    }},
    {"$project": {'rating': '$imdb.rating', "year": "$year"}},
    {"$group": {
        "_id": {
            "year": "$year",
            "rating": "$rating"
        },
        "count": {"$sum": 1}}
    },
    {"$project": {
        "rating": "$_id.rating",
        "year": "$_id.year",
        "count": "$count"
    }},
    {"$sort": {"rating": -1}}
]))

In [ ]:
pp(list(data))

In [ ]:
df = pd.DataFrame(data)
df = df[['count', 'rating', 'year']]

In [ ]:
df['year'] = df['year'].apply(int)
counts = df.pivot('rating', 'year', 'count')

In [ ]:
counts

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(counts, linewidths=0.5, ax=ax, cmap="YlGnBu")

In [ ]:
data = list(db.movies.aggregate([
    {"$match": {
        "imdb.rating": {"$exists": "1", "$gte": 0, "$lte": 10},
        "year": {"$exists": "1"},
        "awards.wins": {"$exists": "1"},
        "awards.nominations": {"$exists": "1"},
        "actors": {"$exists": "1"},
    }},
    {"$project": {
        'rating': '$imdb.rating',
        "year": "$year",
        "wins": "$awards.wins",
        "nominations": "$awards.nominations",
        "actors_count": {"$size": "$actors"},
        "genre": {"$arrayElemAt": ["$genres", 0]}
    }}
]))

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df = df[['actors_count','nominations', 'rating', 'wins', 'year', "genre"]]

In [ ]:
df

In [ ]:
sns.set(style="ticks")
g = sns.pairplot(df)